In [1]:
%pip install BeautifulSoup4
%pip install requests

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
try:
    os.mkdir("ids")
except:
    pass

In [1]:
import time
import random
import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

baseUrl="https://coinreaders.com/search.html?submit=submit&search=%EC%9D%B4%EB%8D%94%EB%A6%AC%EC%9B%80&search_exec=all&news_order=1&search_section=all&search_and=1&search_start_day=&search_end_day=20250912&page="

start=856
end=856

for page in range(start,end+1):
    url = f"{baseUrl}{page}"

    # 브라우저 유사 헤더
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/123.0.0.0 Safari/537.36"
        ),
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Encoding": "gzip, deflate, br",
        # 목록 페이지 → 상세 페이지 흐름을 재현하려면 Referer를 실제 이전 페이지로 설정
        "Referer": baseUrl + str(page),
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
    }

    # 세션 + 재시도(403/429/5xx 백오프)
    retry = Retry(
        total=5,
        backoff_factor=1,  # 1,2,4,8,...
        status_forcelist=[403, 429, 500, 502, 503, 504],
        allowed_methods=["GET", "HEAD", "OPTIONS"]
    )
    adapter = HTTPAdapter(max_retries=retry)

    with requests.Session() as s:
        s.headers.update(headers)
        s.mount("https://", adapter)
        s.mount("http://", adapter)

        # 너무 빠른 요청을 피하기 위한 랜덤 지연
        time.sleep(random.uniform(0.8, 1.6))

        res = s.get(url, timeout=15)
        if res.status_code!=200:
            print(f"crawling failed from page {page}!")
            break
    
    soup=BeautifulSoup(res.text, 'html.parser')
    
    ids = [a.get("href") for a in soup.select(".search_result_list_box > dl > dt > a")]
    
    file=open(f"ids/page{page}.txt","w")
    file.write(",".join(ids))
    file.close()
    print(f"page {page} finished!")


page 856 finished!
